In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Adding a classifier on top of the convnet

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

Training the convnet on MNIST images

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=50, batch_size=64)

Epoch 1/50
938/938 [==============================] - 13s 13ms/step - loss: 0.0160 - accuracy: 0.9951
Epoch 2/50
938/938 [==============================] - 12s 13ms/step - loss: 0.0105 - accuracy: 0.9970
Epoch 3/50
938/938 [==============================] - 12s 13ms/step - loss: 0.0099 - accuracy: 0.9976
Epoch 4/50
938/938 [==============================] - 12s 12ms/step - loss: 0.0079 - accuracy: 0.9975
Epoch 5/50
938/938 [==============================] - 12s 13ms/step - loss: 0.0066 - accuracy: 0.9981
Epoch 6/50
938/938 [==============================] - 11s 12ms/step - loss: 0.0060 - accuracy: 0.9980
Epoch 7/50
938/938 [==============================] - 12s 12ms/step - loss: 0.0064 - accuracy: 0.9981
Epoch 8/50
938/938 [==============================] - 12s 13ms/step - loss: 0.0051 - accuracy: 0.9983
Epoch 9/50
938/938 [==============================] - 12s 12ms/step - loss: 0.0037 - accuracy: 0.9988
Epoch 10/50
938/938 [==============================] - 12s 13ms/step - loss: 0.003

In [8]:
test_loss, test_acc = model.evaluate(test_images,test_labels)
test_acc

313/313 [==============================] - 2s 5ms/step - loss: 0.1520 - accuracy: 0.9923


0.9922999739646912

Saving model

In [9]:
model.save('./results/model_6_1.h5')

In [10]:
import pandas as pd
import numpy as np

num_testing = len(test_images)
pred = np.argmax(model.predict(test_images), axis=-1)
sub_df = pd.DataFrame()
sub_df["ImageId"] = list(range(1, num_testing + 1))
sub_df["Label"] = pred

In [11]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

sub_df["True_Label"] = test_labels

In [12]:
sub_df

,ImageId,Label,True_Label
0,1,7,7
1,2,2,2
2,3,1,1
3,4,0,0
4,5,4,4
...,...,...,...
9995,9996,2,2
9996,9997,3,3
9997,9998,4,4
9998,9999,5,5


In [13]:
sub_df[sub_df.Label != sub_df.True_Label]

,ImageId,Label,True_Label
340,341,3,5
445,446,0,6
447,448,9,4
551,552,9,7
582,583,2,8
...,...,...,...
8408,8409,6,8
9664,9665,7,2
9729,9730,6,5
9770,9771,0,5
